In [1]:
import os
import re
import glob
import numpy as np
import pandas as pd
import sklearn.decomposition

In [ ]:
### example snakemake: /hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/DA_colocs_GxC_ATG10_top_snakemake.py 

In [ ]:
# as it is it tests all cis SNPs which is useful to plot a Manhattan, 
# but for box-plots just observing effect sizes not needed

In [2]:
mydir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/"

In [3]:
myfiles = glob.glob(mydir+"*mean_donor_level.csv")
myfiles

['/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/MPHOSPH9-12_123604053_C_T_top10quantile_mean_donor_level.csv',
 '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/SLC35E2-1_1660978_C_A_bottom30quantile_d52_mean_donor_level.csv',
 '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/MPHOSPH9-12_123604053_C_T_bottom10quantile_mean_donor_level.csv',
 '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/MAPT-AS1-17_43925297_C_T_top10quantile_mean_donor_level.csv',
 '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/PRR7-AS1-5_176904279_C_G_top10quantile_mean_donor_level.csv',
 '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/MAST3-19_18275532_G_A_top10quantile_mean_donor_level.csv',
 '/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/SLC35E2-1_1660978_C_A_top10

In [27]:
file = myfiles[24]
file

'/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/SLC35E2-1_1660978_C_A_top30quantile_d52_mean_donor_level.csv'

In [28]:
cat = re.sub("/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/", "", 
             re.sub("_mean_donor_level.csv","",file))
cat

'SLC35E2-1_1660978_C_A_top30quantile_d52'

In [29]:
outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/input_files_"+cat
outdir

'/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/input_files_SLC35E2-1_1660978_C_A_top30quantile_d52'

In [30]:
# sample_mapping_file = os.path.join(outdir, 'samplemapping.tsv')
phenotype_file = os.path.join(outdir, 'phenotypes.tsv')
covariates_file = os.path.join(outdir, 'covariates.tsv')
noise_matrix_file = os.path.join(outdir, 'noise_matrix.tsv')

In [31]:
noise_matrix_file

'/hps/nobackup/stegle/users/acuomo/all_scripts/struct_LMM2/sc_neuroseq/da_coloc_GxC/input_files_SLC35E2-1_1660978_C_A_top30quantile_d52/noise_matrix.tsv'

In [32]:
# eQTL discovery parameters
#number of top expressed genes to test
n_genes = 50000

In [33]:
if not os.path.exists(outdir):
    os.makedirs(outdir, exist_ok=True)

In [34]:
#load map HGNC to ensembl
mapping_df = pd.read_csv('/nfs/leia/research/stegle/dseaton/genomes/hg19/annotation/geneid_mappings/hgnc_symbol2ensembl_gene_id.txt', sep='\t')

In [35]:
phenotype_df = pd.read_csv(file, index_col=0)

In [36]:
phenotype_df.head()

,MIR1302-10,FAM138A,OR4F5,RP11-34P13.7,RP11-34P13.8,AL627309.1,RP11-34P13.14,RP11-34P13.9,AP006222.2,RP4-669L17.10,...,AL590523.1,CT476828.1,PNRC2-1,SRSF10-1,AC145205.1,BAGE5,CU459201.1,AC002321.2,AC002321.1,n_cells
HPSI0114i-bezi_3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.750000,0.000000,...,0.0,0.0,0.000,0.250000,0.0,0.0,0.0,0.0,0.0,4
HPSI0114i-eipl_1,0.0,0.0,0.0,0.034483,0.0,0.034483,0.0,0.0,0.724138,0.000000,...,0.0,0.0,0.000,0.206897,0.0,0.0,0.0,0.0,0.0,29
HPSI0114i-kolf_2,0.0,0.0,0.0,0.000000,0.0,0.022284,0.0,0.0,0.554317,0.013928,...,0.0,0.0,0.000,0.008357,0.0,0.0,0.0,0.0,0.0,359
HPSI0114i-lexy_2,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.555556,0.000000,...,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,18
HPSI0114i-posc_1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.500000,0.000000,...,0.0,0.0,0.125,0.125000,0.0,0.0,0.0,0.0,0.0,8


In [37]:
cell_count_ds = phenotype_df['n_cells']
cell_count_ds

HPSI0114i-bezi_3      4
HPSI0114i-eipl_1     29
HPSI0114i-kolf_2    359
HPSI0114i-lexy_2     18
HPSI0114i-posc_1      8
                   ... 
HPSI1113i-qolg_1     59
HPSI1113i-qorq_1     36
HPSI1113i-vuud_2    260
HPSI1114i-kuul_1    300
HPSI1213i-hehd_1     72
Name: n_cells, Length: 138, dtype: int64

In [38]:
noise_scaling_vector = [1/float(x) for x in cell_count_ds.tolist()]
noise_matrix = np.diag(noise_scaling_vector)

In [39]:
noise_matrix_df = pd.DataFrame(data=noise_matrix, index=cell_count_ds.index, columns=cell_count_ds.index)
noise_matrix_df.to_csv(noise_matrix_file, sep='\t')

In [40]:
cols_to_drop = list(set(phenotype_df.columns) & {'n_cells'})
phenotype_df = phenotype_df.drop(cols_to_drop, axis=1)
phenotype_df.head()

,MIR1302-10,FAM138A,OR4F5,RP11-34P13.7,RP11-34P13.8,AL627309.1,RP11-34P13.14,RP11-34P13.9,AP006222.2,RP4-669L17.10,...,KIR3DL2-1,AL590523.1,CT476828.1,PNRC2-1,SRSF10-1,AC145205.1,BAGE5,CU459201.1,AC002321.2,AC002321.1
HPSI0114i-bezi_3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.750000,0.000000,...,0.0,0.0,0.0,0.000,0.250000,0.0,0.0,0.0,0.0,0.0
HPSI0114i-eipl_1,0.0,0.0,0.0,0.034483,0.0,0.034483,0.0,0.0,0.724138,0.000000,...,0.0,0.0,0.0,0.000,0.206897,0.0,0.0,0.0,0.0,0.0
HPSI0114i-kolf_2,0.0,0.0,0.0,0.000000,0.0,0.022284,0.0,0.0,0.554317,0.013928,...,0.0,0.0,0.0,0.000,0.008357,0.0,0.0,0.0,0.0,0.0
HPSI0114i-lexy_2,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.555556,0.000000,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
HPSI0114i-posc_1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.500000,0.000000,...,0.0,0.0,0.0,0.125,0.125000,0.0,0.0,0.0,0.0,0.0


In [41]:
phenotype_df = phenotype_df.transpose()
phenotype_df.head()

,HPSI0114i-bezi_3,HPSI0114i-eipl_1,HPSI0114i-kolf_2,HPSI0114i-lexy_2,HPSI0114i-posc_1,HPSI0114i-rozh_4,HPSI0114i-vass_1,HPSI0114i-zapk_2,HPSI0115i-aion_2,HPSI0115i-aoxv_1,...,HPSI1014i-tixi_4,HPSI1014i-vils_1,HPSI1113i-bima_1,HPSI1113i-hayt_3,HPSI1113i-podx_1,HPSI1113i-qolg_1,HPSI1113i-qorq_1,HPSI1113i-vuud_2,HPSI1114i-kuul_1,HPSI1213i-hehd_1
MIR1302-10,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
FAM138A,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
OR4F5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
RP11-34P13.7,0.0,0.034483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.016129,0.0,0.005181,0.0,0.0,0.0,0.007692,0.003333,0.0
RP11-34P13.8,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0


In [42]:
gene_names = phenotype_df.index

In [43]:
#number of top expressed genes
selected_genes = list(phenotype_df.mean(axis=1).nlargest(n_genes).index)
phenotype_df = phenotype_df.loc[selected_genes, :]

In [44]:
phenotype_list = phenotype_df.index
mapping_df = mapping_df.query('hgnc_symbol in @phenotype_list')
mapping_df = mapping_df.drop_duplicates(subset=['hgnc_symbol'])
mapping_df = mapping_df.set_index('hgnc_symbol')

In [45]:
# limit only to hgnc symbols that map to ensembl gene IDs
phenotype_df = phenotype_df.loc[mapping_df.index,:]
phenotype_df.index = mapping_df['ensembl_gene_id']
phenotype_df.head()

,HPSI0114i-bezi_3,HPSI0114i-eipl_1,HPSI0114i-kolf_2,HPSI0114i-lexy_2,HPSI0114i-posc_1,HPSI0114i-rozh_4,HPSI0114i-vass_1,HPSI0114i-zapk_2,HPSI0115i-aion_2,HPSI0115i-aoxv_1,...,HPSI1014i-tixi_4,HPSI1014i-vils_1,HPSI1113i-bima_1,HPSI1113i-hayt_3,HPSI1113i-podx_1,HPSI1113i-qolg_1,HPSI1113i-qorq_1,HPSI1113i-vuud_2,HPSI1114i-kuul_1,HPSI1213i-hehd_1
ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000261657,0.25,0.206897,0.32312,0.166667,0.375,0.588235,0.25,0.322772,0.166667,0.336735,...,0.125,0.370968,0.2,0.259067,0.111111,0.237288,0.444445,0.284615,0.283333,0.263889
ENSG00000229483,0.00,0.000000,0.00000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000232849,0.00,0.000000,0.00000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000229558,0.00,0.000000,0.00000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000224394,0.00,0.000000,0.00000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [46]:
phenotype_df.to_csv(phenotype_file, sep='\t')

In [47]:
############ Covariates (15 PCs)

In [48]:
n_pcs = 15
pc_mat = sklearn.decomposition.PCA(n_components=n_pcs).fit_transform(phenotype_df.values.transpose())
pc_df = pd.DataFrame(data=pc_mat, index=phenotype_df.columns, columns=['PC{}'.format(x) for x in range(1,n_pcs+1)])
pc_df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15
HPSI0114i-bezi_3,77.870752,-107.957914,57.564461,-5.925683,-9.320714,-60.916586,19.412594,-28.340195,-47.502348,27.254238,-4.604248,-11.643281,14.384669,10.531234,-8.666035
HPSI0114i-eipl_1,106.006504,264.180586,36.987586,-41.427729,11.610055,-60.545221,-2.703333,12.632616,-21.866766,14.596258,20.917304,-2.367336,-4.379087,7.644935,4.085440
HPSI0114i-kolf_2,-18.021686,-31.280468,-46.996249,-3.495460,-17.929830,-4.975442,12.438661,14.905257,0.003426,-7.794235,1.299368,0.678019,3.223054,1.065571,-2.201589
HPSI0114i-lexy_2,-49.260435,-18.771702,10.792376,11.414830,-17.720825,-11.299748,-2.605722,-9.941838,3.896168,21.442293,4.105041,-1.762415,-2.176029,0.882617,-3.120671
HPSI0114i-posc_1,-130.538462,-127.862359,35.612257,34.357403,5.865258,6.246783,9.533511,-16.662025,3.516137,10.103211,0.250950,5.062253,10.959520,3.748114,7.163120


In [49]:
pc_df.to_csv(covariates_file, sep='\t')